![alt text](marta_Logo_of_the_Metropolitan_Atlanta_Rapid_Transit_Authority.png "MARTA Logo")

<h1 align=center><font size = 5>Segmenting and Clustering MARTA Rail Stations in Atlanta</font></h1>

## Abstract

In this notebook, I explore and visualize data for venues around MARTA rail stations in Atlanta. First, I use Foursquare to get  popular venues near each station. Next, I use Foursquare categories hierarchy to assign each venue to a general category, thus significantly reducing data dimensionality. Then I explore primary and general venue categories for each station, and use TF-IDF normalization and LDA algorithm to generate  station labels. I also use K-means, DBSCAN and hierarchical clustering algorithms to group stations into category-related clusters. Finally, I visualize all my results in interactive Folium maps.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px" >

<font size = 3>

<b>[Introduction/Business Problem](#item000)</b>
<p>
<b>[1. MARTA Stations Data](#item100)</b> 
<p>
    [1.1 Wikipedia Data](#item110)    
                                        
    [1.2 Foursquare Venue Data](#item120)      

    [1.3 Foursquare Categories Hierarchy](#item130)  
    
    [1.4 Entries/Day vs Number of Venues](#item140)                                                                                
<p>
<b>[2. MARTA Stations Labels](#item200)</b>   
<p>
    [2.1 Venues with Most Traffic](#item210)   
                                      
    [2.2 Venues Specific for Each Station: TF-IDF](#item220)       
    
    [2.3 Venues Specific for Each Station: TF-IDF LDA](#item230)   
    
    [2.4 MARTA Word Cloud](#item240)                                       
<p>
<b>[3. MARTA Stations Clusters](#item300)</b>   
<p>
    [3.1 K-means](#item310)    

    [3.2 DBSCAN](#item320)  
    
    [3.3 Hierarchical Clustering](#item330)     
    
    [3.4 Agglomerative Hierarchical Clustering](#item340)       
<p>
<b>[4. MARTA Stations Annotated Maps](#item400)</b>    
<p>            
<b>[Results](#item500)</b>   
<p>            
<b>[Discussion](#item600)</b>   
<p>            
<b>[Conclusion](#item700)</b>  
</font>
</div>

# Introduction/Business Problem  <a class="anchor" id="item000"></a>


We are going to explore venues associated with each of 38 MARTA rail stations in Atlanta, Georgia.  

This might help us answer the following questions.

Why people use MARTA?  
What attracts them to each MARTA station and makes them go there?  
Where do they go before\after riding MARTA?   
Where do they spend the most time (and money)?  
Are MARTA stations similar or dissimilar in respect to the venues near them?  
Can we label each MARTA station with venue categories that are specific for this station?  

In our analysis, we will use Foursquare API to get the most popular venues near each MARTA station, TF-IDF normalization and LDA algorithm for labeling each station, k-means and hierarchical clustering for grouping similar stations, and Folium library to visualize our results.

MARTA station names and their coordinates can be extracted from Wikipedia https://en.wikipedia.org/wiki/MARTA_rail

# 1. MARTA Stations Data  <a class="anchor" id="item100"></a>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

## 1.1 Wikipedia Data <a class="anchor" id="item110"></a>   

Atlanta MARTA rail network has a total of 38 stations. In order to segement the stations and explore them, we will essentially need a dataset that contains the list of MARTA stations with the latitude and logitude coordinates of each station. 

Luckily, this data exists for free on Wikipedia: https://en.wikipedia.org/wiki/MARTA_rail
It can be easily downloaded and converted to Excel file.

In [ ]:
bFoursquare = False # False: read local csv file instead of foursquare.com

In [ ]:
stations = pd.read_excel('MARTA_stations.xlsx')
print(stations.shape)
stations.head()

Now we can create a map of Atlanta with MARTA stations superimposed on top.

We will use geopy library to get the latitude and longitude values of Downtown Atlanta. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>atlanta_explorer</em>, as shown below.

In [ ]:
address = 'Atlanta, GA'

geolocator = Nominatim(user_agent="atlanta_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Atlanta are {}, {}.'.format(latitude, longitude))

Let's visualize MARTA stations setting marker radius proportional to "Entries/Day".

In [ ]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label, num  in zip(stations['Latitude'], stations['Longitude'], stations['Station'], stations['Entries/Day']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=num / 500,
        popup=label,
        color='red',
        fill=True,
        fill_color='#f4a8a7', #'#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  

map_downtown.save('MARTA_map_Entries.html')
map_downtown

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the station.

## 1.2 Foursquare Venue Data <a class="anchor" id="item120"></a>   

Next, we are going to start utilizing the Foursquare API to explore the stations and segment them.

#### Define Foursquare Credentials and Version

CLIENT_ID = '???????????' # your Foursquare ID  
CLIENT_SECRET = '??????????????' # your Foursquare Secret  
VERSION = '20180605' # Foursquare API version  

In [ ]:
# @hidden_cell


#### Let's explore the first station in our dataframe

Get the neighborhood's name.

In [ ]:
ista = 33
stations.loc[ista, 'Station']

Get the neighborhood's latitude and longitude values.

In [ ]:
station_latitude = stations.loc[ista, 'Latitude']
station_longitude = stations.loc[ista, 'Longitude'] 
station_name = stations.loc[ista, 'Station'] 
print('Latitude and longitude values of {} are {}, {}.'.format(station_name, 
                                                               station_latitude, 
                                                               station_longitude))

Now, let's get the top 100 venues within a radius of 500 meters from this station

First, let's create the GET request URL. 

In [ ]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    station_latitude, 
    station_longitude, 
    radius, 
    LIMIT)

Send the GET request and convert json data into a *pandas* dataframe. All the information is in the *items* key. 

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
if bFoursquare:
    # query
    results = requests.get(url).json()

    # JSON
    venues = results['response']['groups'][0]['items']

    # flatten JSON
    nearby_venues = json_normalize(venues) 

    # filter columns
    filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
    nearby_venues.head()

It seems Foursquare's API has a 100 result limit. We can get more results by using a nearby point. The results may overlap, so we will need to filter the duplicates out and keep changing the point by a tiny margin till the number of unique results stops growing or reaches specified LIMIT. We will leave it beyond the scope of this project for right now, and proceed with results limited to 100.

Let's create a function to repeat the same process to all MARTA stations 

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue ID', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we run the above function on each neighborhood and create a new dataframe called *venues_primary*.

Let's save venues_primary to a backup csv file so that we could re-use it later 

In [ ]:
if bFoursquare:
    venues_primary = getNearbyVenues(names=stations['Station'],
                                   latitudes=stations['Latitude'],
                                   longitudes=stations['Longitude']
                                  )
    venues_primary.to_csv("MARTA_venues.csv",index=False) 
    print(venues_primary.shape)
    print(venues_primary.groupby('Station').count()) #venues were returned for each station
    venues_primary.head()

Let's read venues_primary from csv file

In [ ]:
venues_primary = pd.read_csv("MARTA_venues.csv")
stations = pd.read_excel('MARTA_stations.xlsx')
print(venues_primary.shape)
venues_primary.head()

In [ ]:
venues_count = venues_primary[['Station','Venue']].groupby('Station').count().reset_index().set_index('Station')

Let's visualize MARTA stations setting marker radius proportional to "Venues".

In [ ]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label, num, cnt  in zip(stations['Latitude'], stations['Longitude'], stations['Station'], stations['Entries/Day'], venues_count['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=cnt / 5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  

map_downtown.save('MARTA_map_Venues.html')
map_downtown

Now we have all data ready for our analysis,

## 1.3 Foursquare Categories Hierarchy <a class="anchor" id="item130"></a>    

Let's find out how many unique venues we have found. Some stations are located close to each other and can share some venues. This is OK for the purposes of our analysis, but let's check it.

In [ ]:
tmp = venues_primary.groupby(['Venue Latitude','Venue Longitude']).size().reset_index().rename(columns={0:'count'})
print('There are {} uniques venues out of total {}.'.format(tmp.shape[0],venues_primary.shape[0]))

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(venues_primary['Venue Category'].unique())))

Let's encode our categories using One Hot method

In [ ]:
def one_hot_encoding(venues):
    # one hot encoding
    onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    onehot['Station'] = venues['Station'] 

    # move Station column to the first column
    fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
    onehot = onehot[fixed_columns]
    
    return onehot

In [ ]:
primary_onehot = one_hot_encoding(venues_primary)
print(primary_onehot.shape)
primary_onehot.head()

Notice that each row has exactly one non-zero value. Let's now group them by Station.

In [ ]:
primary = primary_onehot.groupby('Station').sum().reset_index()
primary

#### Extract General Categories 

Our data with 223 primary categories is very sparse. Let's get some more general categories so that we could merge, e.g. Airport Service and Airport Lounge into one general category Airport. We can use Foursquare categories hierarchy tree for this.

In [ ]:
categories_list = primary.columns.tolist()
for x in range(len( categories_list)): 
    print(categories_list[x])

Get Foursquare categories hierarchy tree. 

In [ ]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)

In [ ]:
categories_tree = requests.get(url).json()
categories_tree['response']

We need a function to get tree path given a category name. It will return a list with category name in the very last element, and its root category in the first (0th) element.

In [ ]:
def get_category_path(json_tree, target_name, path):
    for node in json_tree:                    
        if type(node) == dict:
            node_name = node.get("name")
            #print("path=", path + [node_name])  
            
            for key,value in node.items():
                if key == "name":
                    #print(key," = ",value)
                    if value == target_name:
                        #print("FOUND ", key," = ",value)
                        return path + [target_name]
                elif key == "categories":
                    check_child = get_category_path(value, target_name, path + [node_name])
                    if check_child:
                        return check_child

print(get_category_path(categories_tree['response']['categories'], 'Aquarium', []))
print(get_category_path(categories_tree['response']['categories'], 'Art Museum', []))
print(get_category_path(categories_tree['response']['categories'], 'Asian Restaurant', []))
print(get_category_path(categories_tree['response']['categories'], 'Japanese Restaurant', []))
print(get_category_path(categories_tree['response']['categories'], 'Nightclub', []))
print(get_category_path(categories_tree['response']['categories'], 'Whisky Bar', []))
print(get_category_path(categories_tree['response']['categories'], 'Women\'s Store', []))
print(get_category_path(categories_tree['response']['categories'], 'Pharmacy', []))
print(get_category_path(categories_tree['response']['categories'], 'Gas Station', []))
print(get_category_path(categories_tree['response']['categories'], 'Park', []))
print(get_category_path(categories_tree['response']['categories'], 'Yoga Studio', []))
print(get_category_path(categories_tree['response']['categories'], 'Chiropractor', []))
print(get_category_path(categories_tree['response']['categories'], 'Event Space', []))
print(get_category_path(categories_tree['response']['categories'], 'College Basketball Court', []))
print(get_category_path(categories_tree['response']['categories'], 'Residential Building (Apartment / Condo)', []))
print(get_category_path(categories_tree['response']['categories'], 'Airport Lounge', []))
print(get_category_path(categories_tree['response']['categories'], 'Light Rail Station', []))

Now we are ready to extract general category for each primary category

In [ ]:
categories_dict = {}
general_list = categories_list.copy()
for x in range(len(categories_list)): 
    a = get_category_path(categories_tree['response']['categories'], categories_list[x], [])
    if a:
        if len(a) >= 2:
            a = a[0]; # immediate parent: a[-2];
        else:
            a = a[0]; # return itself
        print(a)
        categories_dict[categories_list[x]] = a
    else:
        a = ""
    print(categories_list[x], " --> ", a)
    general_list[x] = a

In [ ]:
print(len(np.unique(categories_list)))
print(len(np.unique(general_list)))

Let's change category names in venues_primary["Venue Category"] using categories_dict.

Original categories before renaming/merging:

In [ ]:
venues_primary.head()

Categories after renaming into 9 general categories:

In [ ]:
venues_general = pd.read_csv("MARTA_venues.csv")
venues_general['Venue Category'] = venues_general['Venue Category'].replace(categories_dict)
venues_general.head()

Let's apply One Hot encoding to this data

In [ ]:
general_onehot = one_hot_encoding(venues_general)
print(general_onehot.shape)
general_onehot.head()

In [ ]:
general = general_onehot.groupby('Station').sum().reset_index()
general

Compared to the table with 223 primary categories, this table with only 9 columns is much easier to interprete. 

In [ ]:
general.sum()

In [ ]:
general.describe()

In [ ]:
general[general['College & University'] > 0]

In [ ]:
general[general['Residence'] > 0]

In [ ]:
general[general['Professional & Other Places'] > 0]

Surprisingly, there are not so many Professional & Other Places venues in Foursquare results. Interestingly, tiny Kensington is also in this list, together with Buckhead and Peachtree Center. This is because Kensington's Utley Chiropractic (Utley Chiropractic & Wellness Center, to be precise) happens to fall into Professional & Other Places category.

Anyhow, Food, Shop & Service and Travel & Transport categories dominate areas around all MARTA stations in Atlanta.

In [ ]:
ax = general.plot(kind='bar', stacked=True, figsize=(18.5, 10.5), cmap="Set3") 
ax.set_ylabel('Number of Venues')
plt.xticks(np.arange(38), list(general['Station']))
plt.savefig('MARTA_stacked_general.png')
plt.show()

## 1.4 Entries/Day vs Number of Venues <a class="anchor" id="item140"></a>   

Now let's plot Entries/Day and number of venues for each station.

In [ ]:
def split_it(name):
    return name.split('/')[0];
split_it("Inman Park/Reynoldstown")

In [ ]:
# add number of venues to stations
merged = stations
merged = merged.join(venues_primary[['Station','Venue']].groupby('Station').count().reset_index().set_index('Station'), on='Station')
merged = merged.drop(['Latitude','Longitude'], axis=1)
#merged['Station'] = merged['Station'].apply(split_it) 
merged

In [ ]:
def plot_scatter_entries_vs_venues(merged, ihighlights=[], name=""):
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Number of Venues', fontsize = 10)
    ax.set_ylabel('Entries/Day', fontsize = 10)
    ax.set_title('MARTA Stations', fontsize = 15)
    ax.grid()
    
    targets = ['no parking', 'free daily', 'free daily & long-term']
    itargets = [0, 1, 2]
    markers = ['o', '^', 's']
    marker_colors = ['b', 'm', 'r']

    if len(ihighlights) == merged.shape[0]:
        # Interprete ihighlights as cluster labels 
        kclusters = np.unique(ihighlights).shape[0]
        xs = np.arange(kclusters)
        ys = [i + xs + (i*xs)**2 for i in range(kclusters)]
        colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
        rainbow = [matplotlib.colors.rgb2hex(i) for i in colors_array]    
        
    for target, color, mark in zip(itargets,marker_colors,markers):
        indicesToKeep = (merged['Parking'] == target)
        ax.scatter(merged.loc[indicesToKeep, 'Venue']
                   , merged.loc[indicesToKeep, 'Entries/Day']
                   , c = color
                   , marker = mark
                   , s = 50)        
    ax.legend(targets)
    
    ix = merged.columns.get_loc("Venue")
    iy = merged.columns.get_loc("Entries/Day")
    for i in range(merged.shape[0]):
        if len(ihighlights) == merged.shape[0]:
            # Interprete ihighlights as cluster labels 
            #ax.scatter(merged.iloc[i,ix], merged.loc[i,iy], c = rainbow[ihighlights[i]-1], marker = 'o', s = 50)        
            ax.annotate(split_it(merged.iloc[i,0]), xy=(merged.iloc[i,ix]+1, merged.iloc[i,iy]), color=rainbow[ihighlights[i]-1])
        else:
            if i in ihighlights:
                ax.annotate(split_it(merged.iloc[i,0]), xy=(merged.iloc[i,ix]+1, merged.iloc[i,iy]), color="black")
            else:
                ax.annotate(split_it(merged.iloc[i,0]), xy=(merged.iloc[i,ix]+1, merged.iloc[i,iy]), color="grey")
    
    plt.xlim(0,120)
    plt.ylim(0,20000)

    # Separator of no-parking from free parking station
    plt.plot([26,26], [0,20000], color='red')
    
    # Save and show
    plt.savefig('MARTA_Entries_vs_Venues_'+name+'.png')
    plt.show()

In [ ]:
#plot_scatter_entries_vs_venues(merged, [], '')
plot_scatter_entries_vs_venues(merged, [3,6,10,11,17,19,22,23,26,29,33], 'Professional')

This plot shows 3 groups of stations: no parking (blue), with free daily parking (magenta) and with free daily & long-term parking (red). Stations with Professionl Services are shown in bold. 

Some observations:

- Number of venues is not correlated with Entries/Day, but is related to free parking. 

- A simple rule "Number of Venues <= 26" surprisingly well separates stations with parking from those with no parking. There are only 3 exceptions with parking (Dunwoody, Lindbergh Center, Inman Park) and 2 exceptions with no parking (Bankhead and Vine City). Lenox is near the boundary. It seems that MARTA stations with parking are mostly used by park-and-ride commuters. 

- Indian Creek, Kensington, College Park and North Springs have many Entries/Day, but a very small number of venues. Indian Creek has only 3 venues: Bus Station,Light Rail Station,Park. Kensington has 4 venues: Gas Station, Discount Store, Pharmacy, Chiropractor

- Five Points, which is a transit point between all four MARTA Rail lines, has the highest number of Entries per Day of all MARTA stations, but only 33 venues. 

- Buckhead, Midtown and Peachtree Center have average to high Entries/Day, and are surrounded with very many venues. There are also smaller stations like this, e.g. King Memorial, Dome, Dunwoody, Decatur, North Avenue, Arts Center, Lindbergh Center. All these stations seem to be located in dense areas with many different businesses.

- Bankhead and West Lake are very small both in number of venues and in Entries/Day. Bankhead has 3 venues: Gas Station, Park, Ice Cream Shop. West Lake has 6 venues: 2 Gas Stations, Light Rail Station, Bus Station, Wings Joint, Café

In [ ]:
d0 = merged.loc[(merged['Parking'] == 0), 'Venue'].as_matrix()
d1 = merged.loc[(merged['Parking'] != 0), 'Venue'].as_matrix()
data_to_plot = [d1, d0]

mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)
ax.boxplot(data_to_plot)
plt.title('Number of venues around MARTA stations')
ax.set_xticklabels(['free parking','no parking'])
plt.savefig('MARTA_Venues_boxplot.png')
plt.show()

# 2 MARTA Stations Labels <a class="anchor" id="item200"></a>   

## 2.1 Venues with Most Traffic <a class="anchor" id="item210"></a>   

Next, let's group venues by station and take the mean of the frequency of occurrence of each category

In [ ]:
general = general_onehot.groupby('Station').mean().reset_index()
print(general.shape)
general.head()

In [ ]:
primary = primary_onehot.groupby('Station').mean().reset_index()
print(primary.shape)
primary.head()

Let's print each station along with the top 5 most common venues

In [ ]:
def print_top_categories(grouped=primary, num_top_venues=5):
    for hood in grouped['Station']:
        print("----"+hood+"----")
        temp = grouped[grouped['Station'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp[temp.freq > 0] # print inly non-zero frequencies
        temp = temp.round({'freq': 2})
        num_top = np.min([num_top_venues,len(temp)])
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top))
        print('\n')

In [ ]:
print_top_categories(grouped=general, num_top_venues=5)

In [ ]:
print_top_categories(grouped=primary, num_top_venues=5)

Let's write functions to sort the venues in descending order and display the top 10 venues for each station in a more compact tabular form.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    tmp = row_categories_sorted.index.values[0:num_top_venues]
    for i in range(num_top_venues):
        if row_categories_sorted[i] <= 0:
            tmp[i] = '-' 
    return tmp

In [ ]:
def top_venues(grouped, num_top_venues, subset, bshorten_names=False):

    num_top = np.min([num_top_venues, grouped.shape[1]-1])
    
    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Station']
    for ind in np.arange(num_top):
        try:
            columns.append('{}{} Most Common Category'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Category'.format(ind+1))

    # create a new dataframe
    venues_sorted = pd.DataFrame(columns=columns)
    venues_sorted['Station'] = grouped['Station']
    
    # shorten station names if required
    if bshorten_names:
        venues_sorted['Station'] = venues_sorted['Station'].apply(split_it)

    # get sorted venues for each row
    if not subset:
        for ind in np.arange(grouped.shape[0]):
            venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top)
    else:
        for ind in np.arange(grouped.shape[0]):
            if ind in subset:
                venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top)
        venues_sorted = venues_sorted.loc[subset]
        
    return venues_sorted

In [ ]:
general_top = top_venues(general, 10, [])
general_top.head()

In [ ]:
primary_top = top_venues(primary, 10, [])
primary_top.head()

This table shows most popular venues, with largest foot traffic.  We can see that these are mostly fast food eateries and coffee shops.

Let's have a closer look at some stations

In [ ]:
top_venues(general, 10, [0,1,6,17,19,27])

In [ ]:
top_venues(primary, 10, [0,1,6,17,19,27])

## 2.2 Venues Specific for Each Stations: TF-IDF <a class="anchor" id="item220"></a>   

Let's now see what makes each MARTA station special and distinct from other stations.

We can do it by using Term Frequency-Inverse Document Frequency (TF-IDF) and similar techniques that are widely used in text processing for normalizing word counts in a corpus of N documents. In our case "word" is a category, and "document" is a station. Each station ("document") can be viewed as a bag of categories ("words") of all venues near that station. 

- Term Frequency (TF) = (Number of times a word w appears in a document d)/(Total number of words in the document d)

- Inverse Document Frequency (IDF) = -log(n/N), where, N is the total number of documents in the corpus and n is the number of documents a word w has appeared in. The IDF of a rare word is high, whereas the IDF of a frequent word is likely to be low.

- We calculate TF-IDF value of a word w in document d as = TF * IDF

Due to the IDF term, TF-IDF method is expected to heavily penalize common words/categories like "Coffee Shop" but assign greater weight to categories like "Airport Lounge" and "Aquarium" that are more distinct and distinguish each station from the other 38 stations.

Let's normalize our data using TF-IDF 

In [ ]:
def normalize_tfidf(primary_tfidf):
    # Drop names
    tmp = primary_tfidf.loc[:, primary_tfidf.columns != 'Station']
    
    # Devide by the row sum  
    TF = tmp.div(tmp.sum(axis=1), axis=0)
    
    # Count non zeroes in each column
    IDF = -np.log(tmp.astype(bool).sum(axis=0, skipna = True) / tmp.shape[0])
    
    # TF * IDF
    tmp = TF * IDF
    
    # Devide by the row sum again so that all values looked like probabilities
    tmp = tmp.div(tmp.sum(axis=1), axis=0)
   
    # Add names back
    primary_tfidf = pd.concat([primary_tfidf[["Station"]], tmp], axis=1)
    
    return primary_tfidf

In [ ]:
general_tfidf = normalize_tfidf(general_onehot.groupby('Station').sum().reset_index()) 
print(general_tfidf.shape)
general_tfidf.head()

In [ ]:
primary_tfidf = normalize_tfidf(primary_onehot.groupby('Station').sum().reset_index()) 
print(primary_tfidf.shape)
primary_tfidf.head()

In [ ]:
print_top_categories(grouped=general_tfidf, num_top_venues=5)

In [ ]:
print_top_categories(grouped=primary_tfidf, num_top_venues=5)

In [ ]:
general_top_tfidf = top_venues(general_tfidf, 10, [])
general_top_tfidf.to_csv('MARTA_general_top.csv')
general_top_tfidf.head()

In [ ]:
primary_top_tfidf = top_venues(primary_tfidf, 10, [])
general_top_tfidf.to_csv('MARTA_primary_top.csv')
primary_top_tfidf.head()

In [ ]:
top_venues(general_tfidf, 10, [0,1,6,17,19,27])

In [ ]:
top_venues(primary_tfidf, 10, [0,1,6,17,19,27])

Notice the difference in the top 10 categories for each station, especially for Airport, Arts Center and Georgia State. Fast food venues are now rated lower for all stations and categories like Airport Service and Performing Arts Venue went to the top. Even for Five Points fast food places gave way to less common Hookah Bar and Cuban Restaurant. Compared to mean frequencies, TF-IDF sorting is much more suitable for labeling\annotating stations. 

Notice also that TF-IDF labels based on primary categories are very specific, e.g. Cuban Restaurant instead of general Food. 

Another way to view this data is to merge all categories columns in one string

In [ ]:
def top_venues_text(top):
    top_text = top.iloc[:,1:].apply(lambda row: ', '.join(row.values.astype(str)), axis=1).reset_index()
    top_text.insert(0, 'Station', list(top['Station']))
    top_text.reset_index().set_index('Station')
    top_text = top_text.drop(['index'], axis=1)
    top_text.columns = ['Station','Top10']
    top_text['Top10'] = top_text['Top10'].str.replace(', -', '')
    return top_text   

In [ ]:
primary_top10 = top_venues_text(primary_top)
primary_top10.to_csv("MARTA_primary_top10.csv")
primary_top10

In [ ]:
primary_top10_tfidf = top_venues_text(primary_top_tfidf)
primary_top10_tfidf.to_csv("MARTA_primary_top10_tfidf.csv")
primary_top10_tfidf

## 2.3 Venues Specific for Each Stations: TF-IDF LDA<a class="anchor" id="item230"></a> 

Let's use LDA topic modelling technique to label the stations based on their primary categories

First, let's merge all primary categories names and get text "documents" that descrcibe each station

In [ ]:
primary_list = venues_primary.groupby(['Station'])['Venue Category'].apply(list).reset_index()
primary_list = primary_list.drop('Station', 1)
primary_list = primary_list.T.squeeze()
primary_list

In [ ]:
primary_text = venues_primary.groupby(['Station'])['Venue Category'].apply(' '.join).reset_index()
primary_text

LDA usually require some text preprocessing (tokenizing, lemmatizing, stemming, etc.). We will use nltk and gensim libraries for this 

In [ ]:
import nltk
nltk.download('wordnet')

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
doc_sample = primary_text.iloc[0,1]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

Now we have 2 options: either consider each document consisting of primary categories (and use primary_list) or break it down to the word level (and use preprocessed primary_text)

In [ ]:
if True:
    primary_docs = primary_text['Venue Category'].map(preprocess)
else:
    primary_docs = primary_list
primary_docs

In [ ]:
dictionary = gensim.corpora.Dictionary(primary_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in primary_docs]

bow_doc = bow_corpus[0]
for i in range(len(bow_corpus[0])):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc[i][0], dictionary[bow_doc[i][0]], bow_doc[i][1]))

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
def lda_topics_words(lda_model):
    lda_topics = pd.DataFrame(columns=['Topic','Words']) 
    lda_topics.set_index('Topic')
    for idx, topic in lda_model.print_topics(-1):
        lda_topics = lda_topics.append({'Topic':'Topic'+str(idx), 'Words':topic}, ignore_index=True)
    return lda_topics

Running LDA using word counts (bag of words)

In [ ]:
np.random.seed(1234)
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
lda_topics = lda_topics_words(lda_model)
lda_topics

Running LDA using TF-IDF

In [ ]:
np.random.seed(1234)
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
lda_topics_tfidf = lda_topics_words(lda_model_tfidf)
lda_topics_tfidf.to_csv("MARTA_lda_topics.csv")
lda_topics_tfidf

Let's use LDA TF-IDF model to classify some station (e.g. 0th, the Airport):

In [ ]:
istation = 1
print(stations.iloc[istation,0])
for index, score in sorted(lda_model_tfidf[bow_corpus[istation]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t Topic{}: {}".format(score, index, lda_model_tfidf.print_topic(index, 10)))

Let's print topic(s) for each station

In [ ]:
def top_venues_lda(lda_model, stations, num_top_topics):
    columns = ['Station']
    for ind in np.arange(num_top_topics):
        columns.append('Topic{}'.format(ind))
    topics_lda = pd.DataFrame(columns=columns)
    topics_lda.set_index('Station')

    for istation, station in enumerate(stations['Station']):
        tmp = lda_model[bow_corpus[istation]]
        di = {'Station': station} 
        for index, score in sorted(lda_model_tfidf[bow_corpus[istation]], key=lambda tup: -1*tup[1]):
            key = 'Topic{}'.format(index)
            di1 = {key: score}
            di = {**di, **di1}
        topics_lda = topics_lda.append(di, ignore_index=True)
    topics_lda.fillna(0, inplace=True)
    return topics_lda

primary_lda_tfidf = top_venues_lda(lda_model_tfidf, stations, 10)
primary_lda_tfidf.to_csv("MARTA_primary_lda_tfidf.csv")
primary_lda_tfidf

## 2.4 MARTA Word Cloud <a class="anchor" id="item240"></a> 

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

def plot_word_cloud(text):
    if False:
        # Create and generate a word cloud image:
        wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
        plt.figure()
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
    else:
        # Use colors from MARTA logo
        stopwords = []
        mask = np.array(Image.open("marta-squarelogo.png"))
        wordcloud_marta = WordCloud(stopwords=stopwords, background_color="white", max_words=1000, mask=mask).generate(doc_sample)
        # create coloring from image
        image_colors = ImageColorGenerator(mask)
        plt.figure(figsize=[7,7])
        plt.imshow(wordcloud_marta.recolor(color_func=image_colors), interpolation="bilinear")
        plt.axis("off")
    
    # store to file and show
    plt.savefig("MARTA_wordcloud.png", format="png")
    plt.show()

In [ ]:
# Start with one station:
doc_sample = primary_text.iloc[0,1]
plot_word_cloud(doc_sample)

In [ ]:
# All stations
tmp = primary_text['Venue Category']
tmp = tmp.T.squeeze()
doc_sample = tmp.str.cat(sep=', ')
plot_word_cloud(doc_sample)

In [ ]:
# Stations with no parking
tmp = primary_text['Venue Category']
tmp = tmp[stations['Parking'] == 0]
tmp = tmp.T.squeeze()
doc_sample = tmp.str.cat(sep=', ')
plot_word_cloud(doc_sample)

In [ ]:
# Stations with parking
tmp = primary_text['Venue Category']
tmp = tmp[stations['Parking'] != 0]
tmp = tmp.T.squeeze()
doc_sample = tmp.str.cat(sep=', ')
plot_word_cloud(doc_sample)

# 3 MARTA Stations Clusters <a class="anchor" id="item300"></a>  

MARTA rail system is not very big. We really don't need any clustering for only 38 stations, 2/3 of which are park-and-ride stations with hardly any venues other than fast food restaurants around them. 

However, for educational purposes let's try to use TF-IDF frequencies to group MARTA stations into some category-related clusters. 

Let's drop station names

In [ ]:
general_data = general_tfidf.drop('Station', 1)
general_data.head()

In [ ]:
primary_data = primary_tfidf.drop('Station', 1)
primary_data.head()

Compute Euclidean distances between stations

In [ ]:
from scipy.stats import wasserstein_distance
from scipy.spatial import distance

def plot_euclidean(data):
    num = data.shape[0]
    w = np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            w[i,j] = distance.sqeuclidean(list(data.iloc[i]), list(data.iloc[j]))

    im = plt.imshow(w, cmap='hot', interpolation='nearest')
    plt.colorbar(im)
    plt.title('Euclidean')
    plt.savefig('MARTA_Euclidean'+str(data.shape[1])+'.png')
    plt.show()

In [ ]:
plot_euclidean(general_data)

In [ ]:
plot_euclidean(primary_data)

As we can see, all distances are very similar and close to 0, except for Bankhead, Indian Creek, Kensington, West Lake stations (indeces: [4, 21, 23, 37]). 

With a distance matrix like this K-means and other clustering algorithms are very likely to separate these 4 stations and consider the other 34 stations as one big cluster. 

In [ ]:
bShowMetrices = False

Let's consider other distances. For example, let's compute pairwise Wasserstein distances between stations

In [ ]:
def plot_wasserstein(data):
    num = data.shape[0]
    w = np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            w[i,j] = wasserstein_distance(list(data.iloc[i]), list(data.iloc[j]))

    im = plt.imshow(w, cmap='hot', interpolation='nearest')
    plt.colorbar(im);
    plt.title('Wasserstein');
    plt.savefig('MARTA_Wasserstein'+str(data.shape[1])+'.png')
    plt.show()

In [ ]:
if bShowMetrices:
    plot_wasserstein(general_data)

In [ ]:
if bShowMetrices:
    plot_wasserstein(primary_data)

Let's compute the Hamming distance between stations, which is simply the proportion of disagreeing components in 2 vectors.

In [ ]:
def plot_hamming(data):
    num = data.shape[0]
    w = np.zeros((num,num))
    for i in range(num):
        for j in range(num):
            w[i,j] = distance.hamming(list(data.iloc[i]), list(data.iloc[j]))

    im = plt.imshow(w, cmap='hot', interpolation='nearest')
    plt.colorbar(im);
    plt.title('Hamming');
    plt.savefig('MARTA_Hamming'+str(data.shape[1])+'.png')
    plt.show()

In [ ]:
if bShowMetrices:
    plot_hamming(general_data)

In [ ]:
if bShowMetrices:
    plot_hamming(primary_data)

## 3.1 K-Means <a class="anchor" id="item310"></a>  

Now we will analyze the K-Means with elbow method. Note that sklearn implementation of K-Means uses only Euclidean distances. 

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

def kmeans_elbow(data):
    distortions = []
    K = range(1,20)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=0).fit(data)
        distortions.append(kmeans.inertia_)

    # Plot the elbow
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.savefig('MARTA_KMeans_Elbow'+str(data.shape[1])+'.png')
    plt.show()

In [ ]:
kmeans_elbow(general_data)

In [ ]:
kmeans_elbow(primary_data)

There's no clear elbow in both plots. It suggests that there might be no meaningful clusters in our data. No elbow might also mean that the algorithm used cannot separate clusters (e.g. when K-means is used for concentric circles, vs DBSCAN). 

Nevertheless, let's run K-Means to cluster the stations and explore these clusters.

In [ ]:
def kmeans(data, k):
    # run k-means clustering
    kmeans = KMeans(n_clusters=k, random_state=0).fit(data)

    # check cluster labels generated for each row in the dataframe
    cluster_labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    print(cluster_labels)
    return cluster_labels

In [ ]:
def print_clusters(grouped, cluster_labels, num_top_venues):
    kclusters = np.unique(cluster_labels).shape[0]
    for cluster in range(kclusters):
        print('-----Cluster ', cluster, '-----') 
        print(top_venues(grouped, num_top_venues, np.where(cluster_labels == cluster)[0].tolist(), True)) # bshorten_names=True

In [ ]:
general_kmeans = kmeans(general_data, 9)
print_clusters(general_tfidf, general_kmeans, 2)

Not bad, actually.

In [ ]:
primary_kmeans = kmeans(primary_data, 9)
print_clusters(primary_tfidf, primary_kmeans, 2)

We can also use Entries/Day, number of venues, Parking and other data for clustering, together with TF-IDF frequencies. We will just need to normalize data matrix before feeding it to Kmeans. 

In [ ]:
from sklearn.preprocessing import StandardScaler

clustering_data = StandardScaler().fit_transform(merged[["Parking","Venue","Entries/Day"]])
kmeans_elbow(clustering_data)

In [ ]:
parking_clusters = kmeans(clustering_data, 5)

Let's play with some other clustering algorithms.

In [ ]:
bPrintClusters = False

## 3.2 DBSCAN <a class="anchor" id="item320"></a>    

In [ ]:
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
sklearn.utils.check_random_state(1000)

sorted(sklearn.neighbors.VALID_METRICS['brute'])

In [ ]:
def dbsc(data):
    # Compute DBSCAN hamming
    dbscan_dataSet = StandardScaler().fit_transform(data)
    db = DBSCAN(eps=0.15, min_samples=1, metric='hamming').fit(dbscan_dataSet)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    cluster_labels = db.labels_
    print(cluster_labels)
    return cluster_labels

In [ ]:
general_dbsc = dbsc(general_data)
if bPrintClusters:
    print_clusters(general_tfidf, general_dbsc, 2)

In [ ]:
primary_dbsc = dbsc(primary_data)
if bPrintClusters:
    print_clusters(primary_tfidf, primary_dbsc, 2)

## 3.3. Hierarchical clustering <a class="anchor" id="item330"></a>    

Unfortunately we can not pass custom distance metrics (e.g. Wasserstein distance) to Sklearn.KMeans. But we can do it with fclusterdata. This function performs hierarchical clustering using the single linkage algorithm, and forms flat clusters using the inconsistency method with t=1.0 as the cut-off threshold.

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import fclusterdata

def fclust(data):
    #fclust = fclusterdata(primary_data, 1.0, metric='euclidean')
    fclust = fclusterdata(data, 1.0, metric=wasserstein_distance)
    cluster_labels = fclust - 1
    print(cluster_labels)
    #print(np.allclose(fclust1, fclust2))
    return cluster_labels

In [ ]:
general_fclust = fclust(general_data)
if bPrintClusters:
    print_clusters(general_tfidf, general_fclust, 2)

In [ ]:
primary_fclust = fclust(primary_data)
if bPrintClusters:
    print_clusters(primary_tfidf, primary_fclust, 2)

## 3.4 Agglomerative Hierarchical Clustering <a class="anchor" id="item340"></a>    

In [ ]:
from sklearn.cluster import AgglomerativeClustering
def aggl(data):
    aggl = AgglomerativeClustering(n_clusters=10, affinity='hamming', linkage='complete')
    cluster_labels = aggl.fit_predict(data) 
    print(cluster_labels) 
    return cluster_labels

In [ ]:
general_aggl = dbsc(general_data)
if bPrintClusters:
    print_clusters(general_tfidf, general_aggl, 2)

In [ ]:
primary_aggl = dbsc(primary_data)
if bPrintClusters:
    print_clusters(primary_tfidf, primary_aggl, 2)


# 4. MARTA Stations Annotated Maps <a class="anchor" id="item400"></a>   

Let's visualize our results using Folium maps.

We can create many different maps, for example:

- general categories clusters and general categories labels
- primary categories clusters and primary categories labels
- general categories clusters and primary categories labels
- parking data (parking/no parking) and general categories labels, etc.

In [ ]:
address = 'Atlanta, GA'

geolocator = Nominatim(user_agent="atlanta_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Atlanta are {}, {}.'.format(latitude, longitude))

In [ ]:
from scipy.stats import mode
def show_map_clusters(top, stations, venues, cluster_labels, bvenues, map_name):
    
    # Merge all station data into one dataframe
    merged = stations
    # add latitude/longitude for each neighborhood
    merged = merged.join(top.set_index('Station'), on='Station')
    # add venues
    merged = merged.join(venues, on='Station')
    # add clustering labels
    #if 'Cluster Labels' in merged.columns:
    merged['Cluster Labels'] = cluster_labels
    #else:
    #    merged.insert(2, 'Cluster Labels', cluster_labels)
   
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

    # set color scheme for the clusters
    # MARTA Orange: #ea7d09 rgb(234, 125, 9)
    # MARTA Blue: #3e78da rgb(62, 120, 218)   
    kclusters = np.unique(cluster_labels).shape[0]
    counts = np.bincount(cluster_labels)
    common_cluster = np.argmax(counts)
    bcommon_cluster = np.max(counts) >= len(cluster_labels)/2
    
    xs = np.arange(kclusters)
    ys = [i + xs + (i*xs)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [matplotlib.colors.rgb2hex(i) for i in colors_array]
    
    # add markers to the map
    markers_colors = []
    for lat, lon, poi, top1, top2, cluster, num, cnt in zip(merged['Latitude'], merged['Longitude'], merged['Station'], 
                merged['1st Most Common Category'], merged['2nd Most Common Category'], merged['Cluster Labels'], 
                merged['Entries/Day'], merged['Venue']):    
        
        if bvenues:
            rad = cnt / 5
        else:
            rad = num / 500
        label = folium.Popup(split_it(poi) + ': ' + top1 + ', ' + top2, parse_html=True)
        tooltip = label
        if bcommon_cluster:
            if cluster != common_cluster: 
                color = 'blue' #  '#3e78da' 
                fill_color = '#3186cc' 
            else:
                color = "red" 
                fill_color = '#f4a8a7'
        else:
            color = rainbow[cluster-1]
            fill_color = rainbow[cluster-1]

        folium.CircleMarker(
            [lat, lon],
            radius=rad,
            popup=label,
            tooltip=tooltip,
            color=color,
            fill=True,
            fill_color=fill_color,
            fill_opacity=0.7).add_to(map_clusters)
    
    map_clusters.save('MARTA_map_'+map_name+str(int(bvenues))+'.html')
    return map_clusters, merged

In [ ]:
print("Number of stations with <  26 venues: ", venues_count[venues_count['Venue'] < 26].count())
print("Number of stations with >= 26 venues: ", venues_count[venues_count['Venue'] >= 26].count())

In [ ]:
general_map, general_merged_tfidf = show_map_clusters(general_top_tfidf, stations, venues_count, general_kmeans, True, "general_kmeans")
general_map

In [ ]:
primary_map, primary_merged_tfidf = show_map_clusters(primary_top_tfidf, stations, venues_count, primary_kmeans, True, "primary_kmeans")
primary_map

In [ ]:
hybrid_map, hybrid_merged_tfidf = show_map_clusters(primary_top_tfidf, stations, venues_count, general_kmeans, True, "hybrid_kmeans")
hybrid_map

In [ ]:
parking_map, parking_merged_tfidf = show_map_clusters(general_top_tfidf, stations, venues_count, 
                                                      list(stations['Parking'].map({0: 0, 1: 1, 2: 1})), True, "general_parking")
parking_map

In [ ]:
parking_map, parking_merged_tfidf = show_map_clusters(primary_top_tfidf, stations, venues_count, 
                                                      list(stations['Parking'].map({0: 0, 1: 1, 2: 1})), True, "primary_parking")
parking_map

In [ ]:
parking_map, parking_merged_tfidf = show_map_clusters(general_top_tfidf, stations, venues_count, 
                                                      parking_clusters, True, "clustering_parking")
parking_map

We can click on each station and see its label\annotation. 

# Results <a class="anchor" id="item500"></a>

-	I used Foursquare to retrieve 223 primary and 9 general categories for 1156 popular venues near 38 MARTA stations
-	I analyzed Entries/Day and number of venues for each MARTA station
-	I discovered that number of nearby venues is strongly correlated with free parking
-	I discovered that MARTA stations can be roughly divided into two major groups: park-and-ride stations, with < 26 venues, and urban stations with no parking and ≥ 26 venues. Three important exceptions are Dunwoody, Lindbergh Center and Inman Park/Reynoldstown located in dense areas. These stations have free parking and many venues nearby
-	I discovered that fast food restaurants, coffee shops and retail shops prevail in areas around all MARTA stations
-	I used TF-IDF and LDA algorithms to extracted venue categories that are most specific\discriminative for each station and used these categories to label\annotate each station
-	I used K-means and other clustering algorithms to group MARTA stations into ~10 clusters based on their venues categories
-	I visualized MARTA data on interactive Folium maps


# Discussion <a class="anchor" id="item600"></a>   

Results for Buckhead, Midtown and Peachtree Center are incomplete, since these stations reached Foursquare API’s venue result limit of 100. We can get more Foursquare results by using nearby points. The results may overlap, so we will need to filter the duplicates out and keep changing the point by a tiny margin till the number of unique venues for each station stops growing or reaches some limit.

MARTA venue analysis can be enriched also by using secondary categories, foot traffic and other detailed information for each venue. This require Foursquare primary calls, so I left it beyond the scope of this small project.

Category merging using Foursquare category tree also has much room for improvement. For example, parent category for "Pharmacy" is "Shop & Service". This makes Medical Center look like any other station.

MARTA station labeling is basically a text document labeling/annotation and topic modeling problem. There are many algorithms for this, with many tunable parameters. It would be interesting, for example, to vary number of topics and build LDA models with different detalization. 

                                               
# Conclusion <a class="anchor" id="item700"></a>  

I analyzed popular venues near 38 MARTA rail stations in Atlanta, Georgia.  

My analysis revealed very low business activity (less than 26 venues) around 18 out of 22 MARTA stations with free parking. Three important exceptions are Dunwoody (I-285/SR 400 interchange), Lindbergh Center (I-85/SR 400 interchange) and Inman Park/Reynoldstown. These three stations provide free daily parking for MARTA passengers, and are located in heavily dense areas with many businesses.

I also discovered that only Civic Center, Lindbergh Center, Inman Park/Reynoldstown, and potentially Buckhead, Midtown and Peachtree Center have Residence venues within walking distance from the train station. Georgia State is the only MARTA station with venues in College & University category.

Fast food restaurants and coffee shops dominate areas around all MARTA stations. Obviously, this is where people spend most of 
their time and money, which is a good news for the fast food companies. 

I used TF-IDF and LDA techniques to label each MARTA station with its discriminative categories/topics. It turned out that there are many Nightlife Spots near MARTA, combined either with Arts & Entertainment, or Food venues.

I created interactive Folium maps of Atlanta with information for each MARTA stations (name, venue categories-based label,  Entries/Day, number of venues, free parking) superimposed on top.

I also discovered that there is a good chiropractic at Kensington station in Atlanta.

### Thank you for reading!

This notebook was created by Nelli Fedorova(https://www.linkedin.com/in/nelli-fedorova-7710b01a/). I hope you found this report interesting and educational. 

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).